In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/"
input_df = pd.read_adls(path + "ln_df_249k.csv", reader = pd.read_csv, dtype = str)
print(input_df.shape)

(249005, 22)


In [10]:
cross_key = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/str_11363_249K_cross_ref.csv")
print(cross_key.shape)

(249000, 4)


In [14]:
cross_key[cross_key["uniqueid"] == "acct_19J6tZIk4jdpEX8o"]

,uniqueid,account,LN_Key1,LN_Key2
19232,acct_19J6tZIk4jdpEX8o,a0106620,FEA8A465FEB248F97117D0B21FDA0DA8,S2_00019233
19359,acct_19J6tZIk4jdpEX8o,a0116638,FEA8A465FEB248F97117D0B21FDA0DA8,S2_00019360


In [11]:
cross_key.head()

,uniqueid,account,LN_Key1,LN_Key2
0,acct_1IaqeBG45P9glmQw,a0068870,07B037DF133464AEEA671B5E8BC039E9,S2_00000001
1,acct_1Huw18DNCme2lTSc,a0174581,AFD8AFEA3E65136D1BFAC27522D8EC5E,S2_00000002
2,acct_1IiFmOBvFw09ic6U,a0219740,F60354932715F9A083E29CFA44AE5403,S2_00000003
3,acct_1JFPAzI8lKdNezrJ,a0188055,16FC78CB5F459E71F25A7FC98D90A5B2,S2_00000004
4,acct_1ISmcQIZ5wPec7JX,a0110218,2F38711812E1BBC9580DBBB3378664CA,S2_00000005


In [3]:
input_df.columns

Index(['Unnamed: 0', 'uniqueid', 'historydate', 'companyname',
       'alternatecompanyname', 'bus_streetaddress1', 'bus_streetaddress2',
       'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website',
       'firstname', 'lastname', 'streetaddress1', 'streetaddress2', 'city',
       'state', 'zip', 'email', 'ip_address', 'performance_indicator'],
      dtype='object')

In [4]:
pd.set_option('display.max_columns', None)
input_df.head(1)

,Unnamed: 0,uniqueid,historydate,companyname,alternatecompanyname,bus_streetaddress1,bus_streetaddress2,bus_city,bus_state,bus_zip,bus_phone,bus_website,firstname,lastname,streetaddress1,streetaddress2,city,state,zip,email,ip_address,performance_indicator
0,0,acct_1GHrCjCbQwUTLFU3,2022-01-31 00:00:00+00:00,Uwani Hub,Uwani.org,2093 Philadelphia Pike,#9212,Claymont,DE,19703,+18322103210,https://uwani.org/,Mojisola,Job,14A Bayo Dejonwo Street,NaN,Maryland Estate,Lagos,100211,mary@howdoyou.tech,197.210.54.73,False


In [5]:
freq(input_df.historydate.str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
historydate,,,,
2021-03,19365.0,0.077770,19365.0,0.077770
2021-04,18728.0,0.075211,38093.0,0.152981
2021-05,17202.0,0.069083,55295.0,0.222064
2021-06,16083.0,0.064589,71378.0,0.286653
2021-07,15938.0,0.064007,87316.0,0.350660
2021-08,16962.0,0.068119,104278.0,0.418779
2021-09,17051.0,0.068477,121329.0,0.487255
2021-10,17376.0,0.069782,138705.0,0.557037
2021-11,18505.0,0.074316,157210.0,0.631353


In [6]:
input_df = input_df[~input_df.historydate.str.slice(0, 7).isin(['Anchora', 'Camaril', 'Newark'])]
print(input_df.shape)

(249000, 22)


In [7]:
input_df = input_df[input_df.historydate.str.slice(0, 7).isin(['2022-01', '2022-02', '2022-03'])]
print(input_df.shape)
freq(input_df.performance_indicator)

(75000, 22)


,Count,Pct,Cuml Count,Cuml Pct
performance_indicator,,,,
False,74362.0,0.991493,74362.0,0.991493
True,638.0,0.008507,75000.0,1.000000


In [ ]:
print()

In [18]:
temp = input_df[["uniqueid", 'performance_indicator']]
temp["count"] = 1
temp_roll = temp.groupby(by = ["uniqueid", "performance_indicator"])['count'].sum().reset_index(name='count')
print(temp_roll.shape)

temp_roll["count_2"] = 1
temp_roll_2 = temp_roll.groupby(by = ["uniqueid"])['count_2'].sum().reset_index(name='count_2')
print(temp_roll_2.shape)

temp_roll_2[temp_roll_2["count_2"] >1]

/tmp/ipykernel_6696/214200790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["count"] = 1


(248315, 3)
(248315, 2)


,uniqueid,count_2


In [24]:
temp_roll[temp_roll["count"] > 1].head()

,uniqueid,performance_indicator,count,count_2
384,acct_14siaOGQrGIpEcyv,False,2,1
443,acct_157AIEIbjCa5R5cG,True,2,1
759,acct_165hHQHJxwJJprCo,False,2,1
2275,acct_18vUlzF6JKIKTwNC,False,2,1
2299,acct_18y4jFIxPAYQwXfm,False,2,1


In [26]:
## looks to be same person with slighly different address
input_df[input_df["uniqueid"] == "acct_165hHQHJxwJJprCo"]

,Unnamed: 0,uniqueid,historydate,companyname,alternatecompanyname,bus_streetaddress1,bus_streetaddress2,bus_city,bus_state,bus_zip,bus_phone,bus_website,firstname,lastname,streetaddress1,streetaddress2,city,state,zip,email,ip_address,performance_indicator
35282,35282,acct_165hHQHJxwJJprCo,2021-04-04 00:00:00+00:00,StrideIT,StrideIT,8 THE GRN STE A,NaN,DOVER,DE,19901,+13322556849,https://codesigncert.com/,Gaurav,Maniar,"A/10/202, Suryacity,","Uttamnagar, Maninagar",Ahmedabad,GJ,380008,webmantra@gmail.com,27.57.165.200,False
133607,133607,acct_165hHQHJxwJJprCo,2021-04-04 00:00:00+00:00,StrideIT,StrideIT,8 THE GRN STE A,NaN,DOVER,DE,19901,+13322556849,https://codesigncert.com/,Gaurav,Maniar,NaN,NaN,NaN,NaN,NaN,webmantra@gmail.com,27.57.165.200,False


In [ ]:
temp = input_df.drop

In [9]:
input_df.to_adls(path + "stripe_file_to_use.parquet")